## Import the necessary libraries

In [16]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [17]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [18]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

## Splitting out the X variable from the target

In [19]:
y = train['sentiment']
X = train['message']

## Turning text into something your model can read

In [20]:
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

## Splitting the training data into a training and validation set

In [21]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.1,shuffle=True,stratify=y, random_state=15)

## Training the model and evaluating using the validation set 

In [22]:
lsvc = LinearSVC()
lsvc = lsvc.fit(X_train, y_train)
lsvc_pred = lsvc.predict(X_val)

## Checking the performance of our model on the validation set

In [23]:
f1_score(y_val, lsvc_pred, average="micro")

0.7648546144121364

In [24]:
from sklearn import metrics

print(metrics.classification_report(y_val, lsvc_pred))

              precision    recall  f1-score   support

          -1       0.74      0.44      0.55       130
           0       0.61      0.47      0.53       235
           1       0.79      0.88      0.83       853
           2       0.77      0.80      0.79       364

    accuracy                           0.76      1582
   macro avg       0.73      0.65      0.68      1582
weighted avg       0.76      0.76      0.76      1582



## Getting our test set ready 

In [25]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [26]:
y_pred = lsvc.predict(test_vect)

In [27]:
test['sentiment'] = y_pred

In [28]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


## Creating an output csv for submission

In [29]:
test[['tweetid','sentiment']].to_csv('testsubmission_8.csv', index=False)